In [2]:
import random
import json
from itertools import chain
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ------------------------------
# 0. Configuración
# ------------------------------
random.seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Usando device:", device)

# ------------------------------
# 1. Catálogo de productos
# ------------------------------
categories = {
    "Frutas y Verduras": ["Manzanas golden","Plátanos","Naranjas","Peras","Kiwis","Tomates","Lechuga","Pepino","Zanahorias","Pimientos","Cebollas","Ajo","Berenjenas","Calabacín","Espinacas","Fresas","Arándanos","Melón","Sandía","Limones"],
    "Carnes": ["Pollo fresco","Ternera filetes","Cerdo lomo","Carne picada","Muslos de pollo","Chuletas de cerdo","Albóndigas","Pechuga de pollo","Filetes de ternera","Jamón cocido"],
    "Pescados": ["Salmón fresco","Merluza","Atún en lata","Bacalao","Trucha","Caballa","Gambas congeladas","Calamares","Mejillones","Anchoas"],
    "Panadería": ["Pan integral","Pan de molde","Baguette","Croissants","Pan de centeno","Panecillos","Bollos","Bagels","Tortillas de trigo","Pan de hamburguesa"],
    "Lácteos": ["Leche entera","Leche semidesnatada","Yogur natural","Queso fresco","Mantequilla","Nata","Yogur griego","Queso curado","Leche sin lactosa","Queso rallado"],
    "Bebidas alcohólicas": ["Cerveza rubia","Cerveza negra","Vino tinto","Vino blanco","Vino rosado","Sidra","Whisky","Ron","Vodka","Ginebra"],
    "Refrescos y aguas": ["Agua mineral","Refresco cola","Refresco naranja","Refresco limón","Zumo de naranja","Zumo de manzana","Tónica","Agua con gas","Refresco de cola cero","Bebida isotónica"],
    "Despensa": ["Arroz largo","Pasta espagueti","Legumbres secas","Aceite de oliva","Vinagre","Azúcar","Sal","Harina","Tomate frito","Salsa de soja","Conservas de atún","Conservas de maíz","Conservas de guisantes","Mermelada","Café molido","Té verde","Chocolate en polvo","Galletas","Miel","Pasta de tomate"],
    "Congelados": ["Croquetas de jamón","Pizza congelada","Hamburguesas congeladas","Verduras congeladas","Helado vainilla","Helado chocolate","Gambas congeladas","Nuggets de pollo","Patatas fritas congeladas","Lasagna congelada"],
    "Limpieza": ["Detergente ropa","Suavizante","Lavavajillas","Limpiador multiusos","Lejía","Esponjas","Fregonas","Paños","Bolsas basura","Guantes de limpieza"],
    "Cuidado personal": ["Champú","Gel de ducha","Jabón de manos","Pasta de dientes","Cepillo de dientes","Desodorante","Acondicionador","Crema facial","Protector solar","Hilo dental"],
    "Snacks y dulces": ["Patatas fritas","Chocolate negro","Chocolate con leche","Galletas dulces","Caramelos","Frutos secos","Bollos rellenos","Barritas de cereales","Chicles","Palomitas"]
}

# ------------------------------
# 2. Tipos de compra y distribuciones
# ------------------------------
purchase_types = {
    "Compra familiar": {"distribution": {"Frutas y Verduras":0.10,"Carnes":0.30,"Pescados":0.10,"Panadería":0.10,"Lácteos":0.10,"Bebidas alcohólicas":0.05,"Refrescos y aguas":0.05,"Despensa":0.15,"Congelados":0.05,"Limpieza":0.05,"Cuidado personal":0.05,"Snacks y dulces":0.0}, "size_range": (15,25)},
    "Compra semanal": {"distribution": {"Frutas y Verduras":0.20,"Carnes":0.15,"Pescados":0.10,"Panadería":0.10,"Lácteos":0.10,"Bebidas alcohólicas":0.05,"Refrescos y aguas":0.10,"Despensa":0.30,"Congelados":0.0,"Limpieza":0.05,"Cuidado personal":0.0,"Snacks y dulces":0.05}, "size_range": (10,20)},
    "Compra saludable": {"distribution": {"Frutas y Verduras":0.40,"Carnes":0.10,"Pescados":0.20,"Panadería":0.05,"Lácteos":0.10,"Bebidas alcohólicas":0.0,"Refrescos y aguas":0.05,"Despensa":0.10,"Congelados":0.0,"Limpieza":0.0,"Cuidado personal":0.0,"Snacks y dulces":0.0}, "size_range": (8,15)},
    "Compra rápida": {"distribution": {"Frutas y Verduras":0.05,"Carnes":0.05,"Pescados":0.0,"Panadería":0.20,"Lácteos":0.10,"Bebidas alcohólicas":0.05,"Refrescos y aguas":0.30,"Despensa":0.10,"Congelados":0.0,"Limpieza":0.0,"Cuidado personal":0.0,"Snacks y dulces":0.15}, "size_range": (4,8)},
    "Fiesta de amigos": {"distribution": {"Frutas y Verduras":0.05,"Carnes":0.10,"Pescados":0.05,"Panadería":0.10,"Lácteos":0.10,"Bebidas alcohólicas":0.20,"Refrescos y aguas":0.10,"Despensa":0.10,"Congelados":0.10,"Limpieza":0.0,"Cuidado personal":0.0,"Snacks y dulces":0.20}, "size_range": (8,18)},
    "Compra urgente/congelados": {"distribution": {"Frutas y Verduras":0.05,"Carnes":0.05,"Pescados":0.05,"Panadería":0.05,"Lácteos":0.10,"Bebidas alcohólicas":0.0,"Refrescos y aguas":0.10,"Despensa":0.15,"Congelados":0.50,"Limpieza":0.0,"Cuidado personal":0.0,"Snacks y dulces":0.0}, "size_range": (5,12)}
}

# ------------------------------
# 3. Generar tickets
# ------------------------------
total_tickets = 15000
type_distribution = {"Compra familiar":0.30,"Compra semanal":0.30,"Compra saludable":0.15,"Compra rápida":0.10,"Fiesta de amigos":0.10,"Compra urgente/congelados":0.05}
tickets_per_type = {k:int(total_tickets*v) for k,v in type_distribution.items()}

def generate_ticket(ticket_id, purchase_type):
    type_info = purchase_types[purchase_type]
    size = random.randint(*type_info["size_range"])
    dist = type_info["distribution"]
    categories_list = list(dist.keys())
    weights = list(dist.values())
    ticket_products = []
    for _ in range(size):
        cat = random.choices(categories_list, weights=weights, k=1)[0]
        prod = random.choice(categories[cat])
        ticket_products.append(prod)
    return {"ticket_id": ticket_id, "purchase_type": purchase_type, "products": ticket_products}

all_tickets = []
ticket_counter = 1
for p_type, n_tickets in tickets_per_type.items():
    for _ in range(n_tickets):
        ticket_id = f"T{ticket_counter:06d}"
        ticket = generate_ticket(ticket_id, p_type)
        all_tickets.append(ticket)
        ticket_counter += 1

# Guardar JSONL
with open("mercadona_smartcart_dataset.jsonl", "w", encoding="utf-8") as f:
    for t in all_tickets:
        f.write(json.dumps(t, ensure_ascii=False)+"\n")

Usando device: cuda


In [5]:
all_products = []

for items in categories.values():  # Recorre cada lista de productos
    all_products.extend(items)     # Añade todos los productos a la lista final

print(all_products)

['Manzanas golden', 'Plátanos', 'Naranjas', 'Peras', 'Kiwis', 'Tomates', 'Lechuga', 'Pepino', 'Zanahorias', 'Pimientos', 'Cebollas', 'Ajo', 'Berenjenas', 'Calabacín', 'Espinacas', 'Fresas', 'Arándanos', 'Melón', 'Sandía', 'Limones', 'Pollo fresco', 'Ternera filetes', 'Cerdo lomo', 'Carne picada', 'Muslos de pollo', 'Chuletas de cerdo', 'Albóndigas', 'Pechuga de pollo', 'Filetes de ternera', 'Jamón cocido', 'Salmón fresco', 'Merluza', 'Atún en lata', 'Bacalao', 'Trucha', 'Caballa', 'Gambas congeladas', 'Calamares', 'Mejillones', 'Anchoas', 'Pan integral', 'Pan de molde', 'Baguette', 'Croissants', 'Pan de centeno', 'Panecillos', 'Bollos', 'Bagels', 'Tortillas de trigo', 'Pan de hamburguesa', 'Leche entera', 'Leche semidesnatada', 'Yogur natural', 'Queso fresco', 'Mantequilla', 'Nata', 'Yogur griego', 'Queso curado', 'Leche sin lactosa', 'Queso rallado', 'Cerveza rubia', 'Cerveza negra', 'Vino tinto', 'Vino blanco', 'Vino rosado', 'Sidra', 'Whisky', 'Ron', 'Vodka', 'Ginebra', 'Agua minera